The aim is to understand Backprop for tensors pretty well. Andrej argues that it's very important to avoiding shooting yourself in the foot. He wrote a [blogpost](https://karpathy.medium.com/yes-you-should-understand-backprop-e2f06eab496b).

In [1]:
### content in this cell is the same as previous notebook Makemore MLP Better than ever
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline 

# Read data
names = open("names.txt", "r").read().splitlines()

# Build vocabulary
characters = sorted(list(set(''.join(names))))
str_to_int = {s:i+1 for i, s in enumerate(characters)}
str_to_int['.'] = 0.0
int_to_str = {i:s for s, i in str_to_int.items()}
vocab_size = len(int_to_str)

block_size = 3  # context size

def build_dataset(names):
    X, Y = [], []
    for name in names:
        context = [0] * block_size
        for character in name + ".":
            ix = str_to_int[character]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y, dtype=torch.long)
    print(X.shape, Y.shape)
    return X, Y

import random 
random.seed(42)
random.shuffle(names)
n1 = int(0.8*len(names))
n2 = int(0.9*len(names))

Xtr, Ytr = build_dataset(names[:n1])
Xdev, Ydev = build_dataset(names[n1:n2])
Xte, Yte = build_dataset(names[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


# Function to Compare Gradients

In [3]:
def cmp(s, dt, t):
    """Compares manual gradients to Pytorch gradients."""
    ex = torch.all(dt == t.grad).item()            # Exact gradient
    app = torch.allclose(dt, t.grad)               # Approximate gradient
    maxdiff = (dt - t.grad).abs().max().item()     # Maximum difference
    print(f"{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}")

# Neural Network Initialization

In [4]:
n_emb = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_emb), generator=g)
# Layer 1
W1 = torch.randn((n_emb*block_size, n_hidden), generator=g) * (5/3) / ((n_emb*block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g)  # Just for fun, it is useless due to BatchNorm
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
# Batch norm
bngain = torch.randn((1, n_hidden), generator=g)*0.1 + 1.0
bnbias = torch.randn((1, n_hidden), generator=g)*0.1 

# Some of these parameters are initialized in non-standard ways because sometimes initializing them 
# with all zeros can mask an incorrect implementation of the backward pass

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


# Batching

In [5]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size, ), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

# Forward Pass

NameError: name 'm' is not defined